In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoData, LayersControl, FullScreenControl, Marker, Icon, LayerGroup, GeoJSON, WidgetControl, Heatmap

%matplotlib inline

In [4]:
_tmpA=pd.read_csv('Department_of_Recreation_and_Parks__Facility_and_Park_Information.csv') \
    [['LocationType', 'Location Name', 'GeoLat', 'GeoLong']]

_tmpB=pd.read_csv('Emergency_Preparedness_Locations_-_2018.csv') \
    [['Category', 'Name', 'Latitude', 'Longitude']]

_tmpC=pd.read_json('Schools.json')

_tmpA=_tmpA.iloc[
    np.where(_tmpA.LocationType.isin(
        ['Beaches', 'Dog Parks', 'Equestrian Centers', 'Golf Courses', 'Gardens', 'Open Space', 'Parks']
    ))]

_tmpA=_tmpA.rename(columns=dict(zip(_tmpA.columns, _tmpB.columns)))

df=_tmpB.append(_tmpA, ignore_index=True)
df=df.append(_tmpC, ignore_index=True)

In [5]:
gdf=gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [6]:
gjson=''
with open('Neighborhoods.geojson') as f:
    import json
    gjson=json.load(f)

neighborhoods=gpd.GeoDataFrame.from_features(gjson)
lacity=neighborhoods.geometry.unary_union

In [9]:
gdf=gdf.iloc[np.where(gdf.geometry.within(lacity.convex_hull))]
df=df.iloc[np.where(gdf.geometry.within(lacity.convex_hull))]

In [10]:
icons=[Icon(icon_url='https://raw.githubusercontent.com/twitter/twemoji/master/assets/72x72/{:x}.png'
            .format(ord(i)), icon_size=(24,24)) for i in '🏢🚒👮🚨🌊🐕🐎🌱⛳⛰🌲🎓🏫']
icons=dict(zip(gdf.Category.unique(), icons))#'🏢🚒👮🚨🌊🐕🐎🌱⛳⛰🌲🎓🏫'))

markers={i:[] for i in gdf.Category.unique()}

for idx,row in gdf.iloc[np.where(gdf.geometry.within(lacity.convex_hull))].iterrows():
    markers[row.Category].append(
        Marker(location=(row.Latitude, row.Longitude), draggable=False, title=row.Name, alt=row.Name, icon=icons[row.Category]))

In [21]:
heatmap = Heatmap(
    locations=[(row.Latitude, row.Longitude) for idx,row in gdf.iloc[np.where(gdf.geometry.within(lacity.convex_hull))].iterrows()],
    radius=30,
    min_opacity=0.2
)

heatmap.radius = 30
heatmap.blur = 50
heatmap.max = 0.5
heatmap.gradient = {0.4: 'red', 0.6: 'yellow', 0.7: 'lime', 0.8: 'cyan', 1.0: 'blue'}

In [22]:
neighborhoods_layer=GeoData(geo_dataframe=neighborhoods, style={'color': 'blue', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.0},
                     hover_style={'fillOpacity': 0.1}, name='Neighborhoods')
geo_data=[]#[LayerGroup(layers=(*markers[i],), name=i) for i in df.Category.unique()]

m=Map(
    center=(df.Latitude.mean(), df.Longitude.mean()),
    controls=(LayersControl(), FullScreenControl()),
    layers=(basemap_to_tiles(basemaps.CartoDB.Positron), *geo_data, neighborhoods_layer, heatmap),
    zoom=10)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [3]:
basemaps.CartoDB.Positron

{'url': 'http://c.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png',
 'max_zoom': 20,
 'attribution': '&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> &copy; <a href="http://cartodb.com/attributions">CartoDB</a>',
 'name': 'CartoDB.Positron'}